## 542 Final Project 
#### Factor Correlation Testing
Tyler Lewis


In [6]:
!pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [3]:
# Imports
import pandas as pd

In [4]:
datasets = [
    '/Users/fsl/Documents/GitHub/CPSC540ParlettPelleriti/FinalProject/data/Bitcoin Fear and Greed Dataset.csv',
    '/Users/fsl/Documents/GitHub/CPSC540ParlettPelleriti/FinalProject/data/BTCUSD Funding Rates History.csv',
    '/Users/fsl/Documents/GitHub/CPSC540ParlettPelleriti/FinalProject/data/M2SL Data.xls'
]

In [7]:
dfs = []
for path in datasets: 
    try:
        dfs.append(pd.read_csv(path))
    except:
        dfs.append(pd.read_excel(path))

In [79]:
btc_stats_df = dfs[0]

# Convert 'Date' to datetime
btc_stats_df['Date'] = pd.to_datetime(btc_stats_df['Date'])

# Extract year and week
btc_stats_df['Year'] = btc_stats_df['Date'].dt.year
btc_stats_df['Week'] = btc_stats_df['Date'].dt.isocalendar().week

# Group by year and week to calculate mean and closing values
btc_stats_weekly_agg = btc_stats_df.groupby(['Year', 'Week']).agg({
    'BTC_Closing': ['first', 'mean', 'last'],  # First for Open, last for Close
    'Value': 'mean',
    'BTC_Volume': 'mean'
}).reset_index()

# Flatten the MultiIndex columns
btc_stats_weekly_agg.columns = ['Year', 'Week', 'BTC_First_Price', 'BTC_Weekly_Avg_Price', 'BTC_Close_Price', 'Weekly_Avg_Fear_Greed', 'BTC_Weekly_Avg_Volume']

# Calculate the Open as the previous week's Close
btc_stats_weekly_agg['BTC_Open_Price'] = btc_stats_weekly_agg['BTC_Close_Price'].shift(1)

# For the first week, set the Open as the First Price
btc_stats_weekly_agg.loc[0, 'BTC_Open_Price'] = btc_stats_weekly_agg.loc[0, 'BTC_First_Price']

# Calculate Percent Change (Open/Close) for the current week
btc_stats_weekly_agg['BTC_Weekly_Pct_Change'] = ((btc_stats_weekly_agg['BTC_Close_Price'] - btc_stats_weekly_agg['BTC_Open_Price']) / btc_stats_weekly_agg['BTC_Open_Price']) * 100

# Calculate Lagged Percent Change from the previous week
btc_stats_weekly_agg['BTC_Prior_Week_Pct_Change'] = btc_stats_weekly_agg['BTC_Weekly_Pct_Change'].shift(1)

# Calculate weekly variance in the Fear/Greed Value (between Open day and Close day)
weekly_value_variance = btc_stats_df.groupby(['Year', 'Week']).agg({
    'Value': ['first', 'last']  # Get the first and last value for the week
}).reset_index()

# Flatten the MultiIndex columns
weekly_value_variance.columns = ['Year', 'Week', 'Fear_Greed_Open', 'Fear_Greed_Close']

# Calculate variance between opening and closing Fear & Greed values
weekly_value_variance['Fear_Greed_Weekly_Change'] = weekly_value_variance['Fear_Greed_Close'] - weekly_value_variance['Fear_Greed_Open']

# Merge the variance into the main DataFrame
btc_stats_weekly_agg = pd.merge(btc_stats_weekly_agg, weekly_value_variance[['Year', 'Week', 'Fear_Greed_Weekly_Change']], on=['Year', 'Week'])

# Add the lagged variance from the previous week
btc_stats_weekly_agg['Fear_Greed_Prior_Week_Change'] = btc_stats_weekly_agg['Fear_Greed_Weekly_Change'].shift(1)

# Drop the intermediate 'First_Price' column
btc_stats_weekly_agg = btc_stats_weekly_agg.drop(columns=['BTC_First_Price'])

btc_stats_weekly_agg.head()

,Year,Week,BTC_Weekly_Avg_Price,BTC_Close_Price,Weekly_Avg_Fear_Greed,BTC_Weekly_Avg_Volume,BTC_Open_Price,BTC_Weekly_Pct_Change,BTC_Prior_Week_Pct_Change,Fear_Greed_Weekly_Change,Fear_Greed_Prior_Week_Change
0,2018,1,3742.700439,3742.700439,26.000000,4.661841e+09,3742.700439,0.000000,NaN,0.0,NaN
1,2018,5,8863.302490,8277.009766,27.250000,9.255910e+09,3742.700439,121.150741,0.000000,-6.0,0.0
2,2018,6,8012.144392,8129.970215,30.571429,8.927013e+09,8277.009766,-1.776482,121.150741,20.0,-6.0
3,2018,7,9869.187221,10551.799805,58.142857,7.660939e+09,8129.970215,29.788911,-1.776482,21.0,20.0
4,2018,8,10443.328683,9664.730469,48.857143,7.912631e+09,10551.799805,-8.406806,29.788911,-34.0,21.0


In [80]:
# remove first 10 rows (misc info from export), rename columns
fred_m2_supply_df = dfs[2][10:].rename(
    columns={
        dfs[2].columns[0]: 'observation_date', 
        dfs[2].columns[1]: 'M2SL'
    }
)

# Convert 'observation_date' to datetime
fred_m2_supply_df['observation_date'] = pd.to_datetime(fred_m2_supply_df['observation_date'])

# Calculate percent change from previous month
fred_m2_supply_df['M2_Monthly_Pct_Change'] = fred_m2_supply_df['M2SL'].pct_change() * 100

# Create a date range for every day between min and max dates
date_range = pd.date_range(
    start=fred_m2_supply_df['observation_date'].min(),
    end=fred_m2_supply_df['observation_date'].max(),
    freq='D'
)

# Create a DataFrame with all dates
all_dates_df = pd.DataFrame({'observation_date': date_range})

# Merge with monthly data, forward fill the M2SL values
all_dates_df = all_dates_df.merge(
    fred_m2_supply_df,
    on='observation_date',
    how='left'
)
all_dates_df['M2SL'] = all_dates_df['M2SL'].ffill()
all_dates_df['M2_Monthly_Pct_Change'] = all_dates_df['M2_Monthly_Pct_Change'].ffill()

# Extract year and week
all_dates_df['Year'] = all_dates_df['observation_date'].dt.year
all_dates_df['Week'] = all_dates_df['observation_date'].dt.isocalendar().week

# Group by year and week, taking the first M2SL and pct_change values
weekly_m2_df = all_dates_df.groupby(['Year', 'Week']).agg({
    'M2SL': 'first',
    'M2_Monthly_Pct_Change': 'first'
}).reset_index()

# Sort by year and week
weekly_m2_df = weekly_m2_df.sort_values(['Year', 'Week'])

print("Sample of weekly data with percent change:")
weekly_m2_df.head(20)

Sample of weekly data with percent change:


/var/folders/ns/qk2_mvqx6w3bpj6wspgtt2c40000gn/T/ipykernel_22694/1057718255.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fred_m2_supply_df['M2_Monthly_Pct_Change'] = fred_m2_supply_df['M2SL'].pct_change() * 100
/var/folders/ns/qk2_mvqx6w3bpj6wspgtt2c40000gn/T/ipykernel_22694/1057718255.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_dates_df['M2SL'] = all_dates_df['M2SL'].ffill()


,Year,Week,M2SL,M2_Monthly_Pct_Change
0,2018,1,13869.2,0.779875
1,2018,2,13869.2,NaN
2,2018,3,13869.2,NaN
3,2018,4,13869.2,NaN
4,2018,5,13869.2,0.274709
5,2018,6,13907.3,0.274709
6,2018,7,13907.3,0.274709
7,2018,8,13907.3,0.274709
8,2018,9,13907.3,0.274709
9,2018,10,13966.4,0.424957


In [81]:
funding_df = dfs[1]

# Convert 'Time(UTC)' to datetime
funding_df['Time(UTC)'] = pd.to_datetime(funding_df['Time(UTC)'])

# Extract year and week
funding_df['Year'] = funding_df['Time(UTC)'].dt.year
funding_df['Week'] = funding_df['Time(UTC)'].dt.isocalendar().week

# Group by year and week, then aggregate Funding Rate
weekly_funding = funding_df.groupby(['Year', 'Week']).agg({
    'Funding Rate': 'mean'  # Or 'sum', 'max', etc.
}).reset_index()

weekly_funding.head(30)

,Year,Week,Funding Rate
0,2018,1,0.000100
1,2018,46,0.000012
2,2018,47,-0.000220
3,2018,48,-0.000333
4,2018,49,0.000017
5,2018,50,0.000007
6,2018,51,-0.000006
7,2018,52,-0.000069
8,2019,1,0.000026
9,2019,2,-0.000078


In [82]:
# Merge all three dataframes on Year and Week
merged_df = btc_stats_weekly_agg.merge(
    weekly_m2_df,
    on=['Year', 'Week'],
    how='inner'
).merge(
    weekly_funding,
    on=['Year', 'Week'],
    how='inner'
)

# Sort by Year and Week
merged_df = merged_df.sort_values(['Year', 'Week'])

# Display info about the merge
print("Original dataframe shapes:")
print(f"BTC Stats: {btc_stats_weekly_agg.shape}")
print(f"M2: {weekly_m2_df.shape}")
print(f"Funding: {weekly_funding.shape}")
print("\nMerged dataframe shape:", merged_df.shape)
print("\nFirst few rows of merged data:")
print(merged_df.head())

# Check for any missing values
print("\nMissing values in merged dataframe:")
print(merged_df.isnull().sum())

Original dataframe shapes:
BTC Stats: (273, 11)
M2: (354, 4)
Funding: (230, 3)

Merged dataframe shape: (230, 14)

First few rows of merged data:
   Year  Week  BTC_Weekly_Avg_Price  BTC_Close_Price  Weekly_Avg_Fear_Greed  \
0  2018     1           3742.700439      3742.700439              26.000000   
1  2018    46           5838.651437      5623.540039              36.857143   
2  2018    47           4361.330008      4009.969971              16.285714   
3  2018    48           4072.562116      4139.877930              15.285714   
4  2018    49           3662.343959      3614.234375              13.857143   

   BTC_Weekly_Avg_Volume  BTC_Open_Price  BTC_Weekly_Pct_Change  \
0           4.661841e+09     3742.700439               0.000000   
1           5.281829e+09     6411.270020             -12.286645   
2           6.076281e+09     5623.540039             -28.693137   
3           6.135039e+09     4009.969971               3.239624   
4           5.483781e+09     4139.877930    

In [86]:
merged_df[1:].to_csv('data.csv')